In [26]:
import random

In [27]:
class Card:
    def __init__(self, suit, number):
        # 初始化属性
        self.suit = suit
        self.number = str(number)
        
        # 定义权重字典，用于 compare 函数的逻辑判断
        self.rank_map = {
            '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, 
            '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14
        }

    def show(self):
        """输出格式：花色-点数"""
        print(f"{self.suit}-{self.number}")

    def compare(self, other):
        """
        比较点数大小
        A > K > Q > J > 10 > ... > 2
        """
        # 获取当前对象和对比对象的权重
        my_rank = self.rank_map[self.number]
        other_rank = self.rank_map[other.number]
        
        if my_rank > other_rank:
            return ">"
        elif my_rank < other_rank:
            return "<"
        else:
            return "="

In [28]:
class Deck:
    def __init__(self):
        # 定义花色和点数的顺序
        suits = ['S', 'H', 'C', 'D']
        numbers = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        
        # 按照题目要求的顺序生成 52 张牌
        # 顺序：S-2, H-2, C-2, D-2, S-3...
        self.cards = []
        for num in numbers:
            for s in suits:
                self.cards.append(Card(s, num))

    def validate(self):
        """
        验证牌堆：
        1. 数量必须为 52
        2. 每一张牌必须互不重复
        """
        if len(self.cards) != 52:
            return False
        
        # 使用集合(set)记录出现过的 (花色, 点数)
        seen_cards = set()
        for card in self.cards:
            card_id = (card.suit, card.number)
            if card_id in seen_cards:
                return False  # 发现重复
            seen_cards.add(card_id)
            
        return True

    def shuffle(self):
        """随机打乱牌序"""
        random.shuffle(self.cards)

    def draw(self):
        """从牌顶抽一张（返回 Card 对象并移除）"""
        if len(self.cards) > 0:
            return self.cards.pop(0)  # pop(0) 移除列表第一个元素（牌顶）
        else:
            return None

In [29]:
def run_experiment(times):
    equal_count = 0
    
    for _ in range(times):
        # 1. 初始化
        deck = Deck()
        
        # 2. 洗牌并验证
        deck.shuffle()
        if not deck.validate():
            continue # 验证失败（理论上不应发生）
            
        # 3. 抽第一张牌并验证 (此时 51 张，validate 应返回 False)
        card1 = deck.draw()
        is_valid_after_draw = deck.validate()
        
        # 4. 抽第二张牌并比较
        card2 = deck.draw()
        result = card1.compare(card2)
        
        if result == "=":
            equal_count += 1
            
    frequency = equal_count / times
    return frequency

In [30]:
# --- 测试代码 ---
if __name__ == "__main__":
    card1 = Card("S", "A")   # 黑桃 A
    card2 = Card("D", "10")  # 方块 10
    card3 = Card("H", "10")  # 红桃 10

    # 测试 show 方法
    card1.show()  # 输出: S-A
    card2.show()  # 输出: D-10

    # 测试 compare 方法
    print(f"A vs 10: {card1.compare(card2)}")  # 输出: >
    print(f"10 vs A: {card2.compare(card1)}")  # 输出: <
    print(f"10 vs 10: {card2.compare(card3)}") # 输出: =
    
    deck = Deck()
    
    # 1. 验证初始生成的牌
    print(f"初始验证通过: {deck.validate()}") # 应返回 True
    
    # 2. 查看前 4 张牌（应为 S-2, H-2, C-2, D-2）
    print("牌顶的前4张牌：")
    for i in range(4):
        deck.cards[i].show()
        
    # 3. 洗牌并抽牌
    deck.shuffle()
    print(f"洗牌后验证通过: {deck.validate()}")
    
    print("抽出的牌是：")
    my_card = deck.draw()
    my_card.show()
    
    print(f"抽完一张后剩余牌数: {len(deck.cards)}")

    freq_1k = run_experiment(1000)
    freq_100k = run_experiment(100000)
    theoretical_prob = 3 / 51

    print(f"【实验结果】")
    print(f"理论概率: {theoretical_prob:.6f}")
    print(f"1,000 次实验频率: {freq_1k:.6f}")
    print(f"100,000 次实验频率: {freq_100k:.6f}")

S-A
D-10
A vs 10: >
10 vs A: <
10 vs 10: =
初始验证通过: True
牌顶的前4张牌：
S-2
H-2
C-2
D-2
洗牌后验证通过: True
抽出的牌是：
S-2
抽完一张后剩余牌数: 51
【实验结果】
理论概率: 0.058824
1,000 次实验频率: 0.054000
100,000 次实验频率: 0.058660


In [31]:
# import random
# import time
# from concurrent.futures import ProcessPoolExecutor

# # 假设 Card 和 Deck 类已经定义好了

# def batch_trials(num_samples):
#     """
#     在一个进程内执行多次实验，大幅减少进程间通信开销
#     """
#     local_equal_count = 0
#     # 在进程内初始化一次，避免重复创建不必要的辅助对象
#     deck = Deck() 
    
#     for _ in range(num_samples):
#         # 模拟实验流程
#         deck.__init__() # 快速重置牌堆
#         deck.shuffle()
        
#         c1 = deck.draw()
#         c2 = deck.draw()
        
#         if c1.compare(c2) == "=":
#             local_equal_count += 1
            
#     return local_equal_count

# def run_optimized_experiment(total_times, num_workers=4):
#     # 将总数切分为大块
#     samples_per_worker = total_times // num_workers
#     tasks = [samples_per_worker] * num_workers
    
#     # 补齐余数
#     tasks[-1] += total_times % num_workers
    
#     total_equals = 0
#     # 使用多进程池
#     with ProcessPoolExecutor(max_workers=num_workers) as executor:
#         # map 会分发给各个进程执行 batch_trials
#         results = list(executor.map(batch_trials, tasks))
#         total_equals = sum(results)
        
#     return total_equals / total_times

# if __name__ == "__main__":
#     TOTAL = 100000
#     print(f"开始执行 {TOTAL} 次优化实验...")
    
#     start = time.time()
#     freq = run_optimized_experiment(TOTAL, num_workers=4)
#     end = time.time()
    
#     print(f"耗时: {end - start:.2f} 秒")
#     print(f"实验频率: {freq:.6f}")
#     print(f"理论概率: {3/51:.6f}")